In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
from copulas.multivariate import GaussianMultivariate
import numpy as np
from itertools import combinations
import importlib
import logging
import itertools



In [2]:
Data_folder = os.getcwd() + "/Data"

Data_folder

'/home/trapfishscott/Cambridge24.25/Energy_thesis/Data'

In [3]:
## Import pickls

car_df = pd.read_pickle(Data_folder + "/df_car.pkl")

In [4]:
car_df["TripType"] = list(zip(car_df["TripPurpFrom_B01ID"], car_df["TripPurpTo_B01ID"]))

car_df = car_df.drop(["TripPurpFrom_B01ID", "TripPurpTo_B01ID"], axis=1)

car_df

,DayID,IndividualID,TravDay,JourSeq,TripStart,TripEnd,TripDisExSW,TravelYear,TravelWeekDay_B03ID,TripType
0,2014050554,2014007801,7,1,420.0,425.0,2.0,2014,1,"(3, 1)"
1,2014050554,2014007801,7,3,1080.0,1125.0,10.0,2014,1,"(3, 2)"
2,2014050561,2014007802,7,1,360.0,395.0,13.0,2014,1,"(3, 1)"
3,2014050561,2014007802,7,3,1200.0,1220.0,5.0,2014,1,"(3, 2)"
4,2014050567,2014007803,6,2,1140.0,1155.0,3.5,2014,1,"(2, 3)"
...,...,...,...,...,...,...,...,...,...,...
5342915,2023101345,2023017037,6,1,565.0,580.0,9.3,2024,2,"(3, 2)"
5342916,2023101345,2023017037,6,2,590.0,610.0,7.0,2024,2,"(2, 2)"
5342917,2023101345,2023017037,6,3,620.0,655.0,16.0,2024,2,"(2, 3)"
5342918,2023101346,2023017037,7,1,710.0,750.0,20.0,2024,2,"(3, 2)"


In [5]:
# Trip type by travel day by journey sequence
x = car_df.groupby(["IndividualID", "TravDay", "JourSeq"])[["TripType"]].first()
# Trip type by travel day by journey sequence
x = x.groupby(["IndividualID", "TravDay"])[["TripType"]].agg(list)

x = x.reset_index()

In [6]:
x["SumTrips"] = x["TripType"].apply(lambda x: len(x))

In [7]:
days = np.tile(np.arange(1,8), len(x["IndividualID"].unique()))

individuals = [i for i in x["IndividualID"].unique() for _ in range(7)]

new_df = pd.DataFrame({"IndividualID": individuals,
                       "TravDay": days })

merged_df = new_df.merge(x, on=["IndividualID", "TravDay"], how="left")

merged_df = merged_df.fillna(0)

merged_df["TripType"] = merged_df["TripType"].apply(lambda x: str(x))


In [9]:


print(f"There are {len(merged_df["TripType"].unique())} unique trip sequences.")


There are 5921 unique trip sequences.


In [11]:
merged_df

,IndividualID,TravDay,TripType,SumTrips
0,2002000001,1,0,0.0
1,2002000001,2,0,0.0
2,2002000001,3,0,0.0
3,2002000001,4,0,0.0
4,2002000001,5,0,0.0
...,...,...,...,...
1211583,2023017379,3,"[(3, 1), (1, 3), (3, 2), (2, 3)]",4.0
1211584,2023017379,4,"[(3, 2), (2, 3)]",2.0
1211585,2023017379,5,0,0.0
1211586,2023017379,6,"[(3, 1), (1, 3)]",2.0


In [20]:
# Finding if 1 trip, 2 trips, 3 trips ... per day. How many unique trip sequences in total + probabilities for different kinds of trip sequences

trip_probs = merged_df[["SumTrips", "TripType"]].groupby("SumTrips").value_counts(normalize=True).reset_index()

In [25]:
trip_probs

,SumTrips,TripType,proportion
0,0.0,0,1.000000
1,1.0,"[(3, 2)]",0.359880
2,1.0,"[(2, 3)]",0.358808
3,1.0,"[(3, 1)]",0.137585
4,1.0,"[(1, 3)]",0.094877
...,...,...,...
5916,20.0,"[(3, 2), (2, 3), (3, 2), (2, 2), (2, 3), (3, 2...",0.333333
5917,21.0,"[(3, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2...",0.333333
5918,21.0,"[(3, 2), (2, 2), (2, 2), (2, 2), (2, 3), (3, 2...",0.333333
5919,21.0,"[(3, 2), (2, 3), (3, 2), (2, 2), (2, 2), (2, 3...",0.333333


In [ ]:
## For each trip length we need A population and weights (ie probabilities)



In [ ]:
y = car_df.groupby(["IndividualID", "TravDay"]).count().index

y

MultiIndex([(2002000001, 6),
            (2002000001, 7),
            (2002000002, 1),
            (2002000002, 2),
            (2002000002, 3),
            (2002000002, 4),
            (2002000002, 5),
            (2002000002, 6),
            (2002000002, 7),
            (2002000006, 1),
            ...
            (2023017375, 1),
            (2023017375, 2),
            (2023017375, 3),
            (2023017375, 4),
            (2023017379, 1),
            (2023017379, 2),
            (2023017379, 3),
            (2023017379, 4),
            (2023017379, 6),
            (2023017379, 7)],
           names=['IndividualID', 'TravDay'], length=827492)

In [24]:
x.index

MultiIndex([(2002000001, 6, 1),
            (2002000001, 6, 2),
            (2002000001, 6, 3),
            (2002000001, 6, 4),
            (2002000001, 6, 5),
            (2002000001, 7, 3),
            (2002000001, 7, 4),
            (2002000001, 7, 5),
            (2002000001, 7, 6),
            (2002000002, 1, 1),
            ...
            (2023017379, 3, 3),
            (2023017379, 3, 4),
            (2023017379, 4, 1),
            (2023017379, 4, 2),
            (2023017379, 6, 1),
            (2023017379, 6, 2),
            (2023017379, 7, 1),
            (2023017379, 7, 2),
            (2023017379, 7, 3),
            (2023017379, 7, 4)],
           names=['IndividualID', 'TravDay', 'JourSeq'], length=2576424)